In [1]:
import pandas as pd
from datetime import datetime
import re

In [ ]:
pd.to_datetime(datetime.now().date(), dayfirst=True, format="dd-mm-YYYY")

Timestamp('2024-12-17 00:00:00')

In [2]:
stihiya = {"вода":"металл", "дерево":"вода", "огонь":"дерево", "земля":"огонь", "металл":"земля"}
stvoly_Ke = {"Lu":"Si", "Co":"Liv", "Sp":"Gb", "St":"Kid", "Ht":"Bl"}
vetvi_Ke = {"Lu":"Bl", "Co":"Kid", "Sp":"Th", "St":"Hg", "Ht":"Gb", "Si":"Liv"}
u_sin_pitanie = {'Ht': 'Liv',
                'Si': 'Gb',
                'Hg': 'Liv',
                'Th': 'Gb',
                'Sp': 'Hg',
                'St': 'Th',
                'Lu': 'Sp',
                'Co': 'St',
                'Kid': 'Lu',
                'Bl': 'Co',
                'Liv': 'Kid',
                'Gb': 'Bl'}
u_sin_Ke = {'Hg': 'Bl',
            'Ht': 'Bl',
            'St': 'Liv',
            'Lu': ['Th','Si'],
            'Co': ['Hg', 'Ht'],
            'Kid': 'St',
            'Bl': 'Sp',
            'Liv': 'Co',
            'Gb': 'Lu',
            'Th': 'Kid',
            'Si': 'Kid',
            'Sp':'Gb'}
home_Ke = {'Liv': 'Gb',
            'Gb': 'Liv',
            'Ht': ['Si', 'Th'],
            'Si': ['Ht', 'Hg'],
            'Hg': ['Si', 'Th'],
            'Th': ['Ht', 'Hg'],
            'Sp': 'St',
            'St': 'Sp',
            'Lu': 'Co',
            'Co': 'Lu',
            'Kid': 'Bl',
            'Bl': 'Kid'}


def get_Ke(d, value):
    for k, v in d.items():
        if v == value:
            return k
        elif k == value:
            return v
        

In [3]:
polugodie = pd.read_excel("Полугодия.xlsx", index_col="year")
u_sin = pd.read_excel("У-син.xlsx", index_col="Орган")
season_qi = pd.read_excel("season_qi.xlsx", index_col="Орган")
qi = pd.read_excel("qi.xlsx", index_col="Стихия")
stvoly = pd.read_excel("stvoly.xlsx", index_col="year")
vetvi = pd.read_excel("vetvi.xlsx", index_col="year")
sloy = pd.read_excel("sloy.xlsx", index_col="year")

In [18]:

try:
    birthday = str(pd.to_datetime(input("Введите дату рождения"), dayfirst=True)).split()[0]
    print(birthday)
    year = int(birthday[:4])
except:
    print("Некорректная дата. Попробуйте снова")

if birthday in polugodie.loc[year, "I полугодие"]:
    print(f"I полугодие {year} года")
    polugodie_true = "I полугодие"
    polugodie_false = "II полугодие"
elif birthday in polugodie.loc[year, "II полугодие"]:
    print(f"II полугодие {year} года")
    polugodie_true = "II полугодие"
    polugodie_false = "I полугодие"
else:
    print(f"II полугодие {year-1} года")
    polugodie_true = "II полугодие"
    polugodie_false = "I полугодие"
    year = year-1


1980-03-16
I полугодие 1980 года


In [19]:


df = pd.DataFrame(
    {"Ствол":[stvoly.loc[year, 'Орган'], get_Ke(stvoly_Ke, stvoly.loc[year, 'Орган'])],
    "Ветвь":[vetvi.loc[year, 'Орган'], get_Ke(vetvi_Ke, vetvi.loc[year, 'Орган'])],
    "Слой":[sloy.loc[year, polugodie_true], sloy.loc[year, polugodie_false]],
    "Zang Fu Xu":[" ", None]}
)
df = df.T
df = df.rename(columns={
    0:"Не используем",
    1:"Используем"
})

home_full = {stvoly.loc[year, 'Стихия'], 
             u_sin.loc[stvoly.loc[year, 'Орган'], "Стихия"],
             u_sin.loc[vetvi.loc[year, 'Орган'], "Стихия"],
             u_sin.loc[sloy.loc[year, polugodie_true].split("+")[0], "Стихия"],
             u_sin.loc[sloy.loc[year, polugodie_true].split("+")[1], "Стихия"]}

season_qi_full = { 
             season_qi.loc[stvoly.loc[year, 'Орган'], "Стихия"],
             season_qi.loc[vetvi.loc[year, 'Орган'], "Стихия"],
             season_qi.loc[sloy.loc[year, polugodie_true].split("+")[0], "Стихия"],
             season_qi.loc[sloy.loc[year, polugodie_true].split("+")[1], "Стихия"]}

home_empty = list(set(stihiya.keys()) - home_full)
season_qi_empty = list(set(stihiya.keys()) - season_qi_full)

a = set(u_sin[u_sin["Стихия"].isin(home_empty)].index.to_list())
b = set(season_qi[season_qi["Стихия"].isin(season_qi_empty)].index.to_list())

print(f"Пустой дом по У-СИН: {a} \t\t\t\t ({home_empty})")
print(f"Пустая сезонная ЦИ: {b} \t ({season_qi_empty})")

df.loc["Zang Fu Xu", "Используем"] = a & b
df

Пустой дом по У-СИН: {'Sp', 'St'} 				 (['земля'])
Пустая сезонная ЦИ: {'St', 'Si', 'Sp', 'Ht', 'Hg', 'Liv'} 	 (['огонь', 'земля'])


,Не используем,Используем
Ствол,Co,Liv
Ветвь,Bl,Lu
Слой,Gb+Th,Hg+Liv
Zang Fu Xu,,"{Sp, St}"


In [121]:
# polugodie.to_excel("Полугодия.xlsx")
# u_sin.to_excel("У-син.xlsx")
# season_qi.to_excel("season_qi.xlsx")
# qi.to_excel("qi.xlsx")
# stvoly.to_excel("stvoly.xlsx")
# vetvi.to_excel("vetvi.xlsx")
# sloy.to_excel("sloy.xlsx")

In [20]:
plus = 'ht, hg'
canals_p = re.sub('[:,/.;#$%^&]', ' ', plus).split()
canals_plus = []
for c in canals_p:
    canals_plus.append(c.capitalize())
            
if canals_plus:
    y = []
    for elem in canals_plus:     
        y.append(get_Ke(stvoly_Ke, elem))
        y.append(get_Ke(vetvi_Ke, elem))
        
        if type(u_sin_Ke[elem]) == type(list()):
            y.append(u_sin_Ke[elem][0])
            y.append(u_sin_Ke[elem][1])
        else:
            y.append(u_sin_Ke[elem])
            
        if type(home_Ke[elem]) == type(list()):    
            y.append(home_Ke[elem][0])
            y.append(home_Ke[elem][1])
        else:
            y.append(home_Ke[elem])
    y = [x for x in y if x is not None]

    canals = []

    df_2 = pd.DataFrame(
        columns=list(set(y)),
        index=["Точка трансформации", "Точка качества дома", "Точка сезонной ци", "Точки трансформации сезонной ци"]
    )

    for el in y:
        canals.append(u_sin_pitanie[el])
        try:
            df_2.loc[df_2.index[0], el] = f'{u_sin_pitanie[el]}{qi.loc[u_sin.loc[u_sin_Ke[el], "Стихия"].mode()[0], u_sin_pitanie[el]]}'
        except:
            df_2.loc[df_2.index[0], el] = f'{u_sin_pitanie[el]}{qi.loc[u_sin.loc[u_sin_Ke[el], "Стихия"], u_sin_pitanie[el]]}'
        
        try:
            df_2.loc[df_2.index[1], el] = f'{u_sin_pitanie[el]}{qi.loc[u_sin.loc[u_sin_pitanie[el], "Стихия"].mode()[0], u_sin_pitanie[el]]}'
        except:
            df_2.loc[df_2.index[1], el] = f'{u_sin_pitanie[el]}{qi.loc[u_sin.loc[u_sin_pitanie[el], "Стихия"], u_sin_pitanie[el]]}'

        celll = " "
        cell = " "
        for c in season_qi[season_qi["Стихия"] == stihiya[season_qi.loc[el, "Стихия"]]].index.to_list():
            cell+=(f'{c}{qi.loc[stihiya[season_qi.loc[el, "Стихия"]], c]}, ')
            celll+=(f'{c}{qi.loc[stihiya[stihiya[season_qi.loc[el, "Стихия"]]], c]}, ')  
            canals.append(c)
        df_2.loc[df_2.index[2], el] = cell
        df_2.loc[df_2.index[3], el] = celll 

df_2

,St,Si,Bl,Gb,Th
Точка трансформации,Th3,Gb43,Co11,Bl67,Gb43
Точка качества дома,Th6,Gb41,Co1,Bl66,Gb41
Точка сезонной ци,"Ht8, Sp2,","Ht8, Sp2,","Si8, St36, Liv3, Hg7,","Bl67, Kid7,","Bl67, Kid7,"
Точки трансформации сезонной ци,"Ht9, Sp1,","Ht9, Sp1,","Si5, St41, Liv2, Hg8,","Bl40, Kid3,","Bl40, Kid3,"


In [21]:
minus = 'sp'
canals_p = re.sub('[:,/.;#$%^&]', ' ', minus).split()
canals_minus = []
for c in canals_p:
    canals_minus.append(c.capitalize())

if canals_minus:
    
    df_3 = pd.DataFrame(
        columns=canals_minus,
        index=["Точка трансформации", "Точка качества дома", "Точка сезонной ци", "Точки трансформации сезонной ци"]
    )
    
    canals = []
    for minus in canals_minus:
        canals.append(u_sin_pitanie[minus])
        try:
            df_3.loc[df_3.index[0], minus] = f'{u_sin_pitanie[minus]}{qi.loc[u_sin.loc[u_sin_Ke[minus], "Стихия"].mode()[0], u_sin_pitanie[minus]]}'
        except:
            df_3.loc[df_3.index[0], minus] = f'{u_sin_pitanie[minus]}{qi.loc[u_sin.loc[u_sin_Ke[minus], "Стихия"], u_sin_pitanie[minus]]}'
        
        try:
            df_3.loc[df_3.index[1], minus] = f'{u_sin_pitanie[minus]}{qi.loc[u_sin.loc[u_sin_pitanie[minus], "Стихия"].mode()[0], u_sin_pitanie[minus]]}'
        except:
            df_3.loc[df_3.index[1], minus] = f'{u_sin_pitanie[minus]}{qi.loc[u_sin.loc[u_sin_pitanie[minus], "Стихия"], u_sin_pitanie[minus]]}'

        
        cell = " "
        for c in season_qi[season_qi["Стихия"] == stihiya[season_qi.loc[minus, "Стихия"]]].index.to_list():
            cell+=(f'{c}{qi.loc[stihiya[season_qi.loc[minus, "Стихия"]], c]}, ')
            canals.append(c)
        df_3.loc[df_3.index[2], minus] = cell

        celll = " "
        for c in season_qi[season_qi["Стихия"] == stihiya[season_qi.loc[minus, "Стихия"]]].index.to_list():
            celll+=(f'{c}{qi.loc[stihiya[stihiya[season_qi.loc[minus, "Стихия"]]], c]}, ')  
        df_3.loc[df_3.index[3], minus] = celll 


use = set([df.loc['Ствол', 'Используем'], df.loc['Ветвь', 'Используем'], df.loc['Слой', 'Используем']]) | (df.loc['Zang Fu Xu', 'Используем'])

dnt_use = set([df.loc['Ствол', 'Не используем'], df.loc['Ветвь', 'Не используем'], df.loc['Слой', 'Не используем']]) 

joined = (set(canals) | set(y)) #- dnt_use
print(joined)
df_3

{'Co', 'St', 'Si', 'Lu', 'Bl', 'Gb', 'Th', 'Hg'}


,Sp
Точка трансформации,Hg9
Точка качества дома,Hg8
Точка сезонной ци,"Co3, Lu11,"
Точки трансформации сезонной ци,"Co2, Lu5,"


In [22]:

if (df.loc['Слой', 'Используем'].split('+')[0] in joined) & (df.loc['Слой', 'Используем'].split('+')[1] in joined):
    print(f"""Можем использовать слой: {df.loc['Слой', 'Используем']}""")
    tochki = []
    for i in df_2.index:
        tochki = []
        for c in df_2.columns:
            tochki.extend(re.findall(f"({df.loc['Слой', 'Используем'].split('+')[0]}\d*|{df.loc['Слой', 'Используем'].split('+')[1]}\d*)", df_2.loc[i, c]))
        print('\t',i, set(tochki)) 
else:
    ttt = []
    for i in df_2.index:
        tt = (re.sub(f'({df.loc[df.index[0], "Не используем"]}\d*|{df.loc[df.index[1], "Не используем"]}\d*)', '', df_3.loc[i, df_3.columns[0]]))
        tt = tt.replace(',', '').strip()
        ttt.append(tt)
        print(f"""{i}: {tt}""")

    ttt = [x for x in ttt if len(x) > 0]
    
    if ttt:
        print(f"""Наиболее эффективная точка для питания: {ttt[0]}""")
    else:
        print(f"""Подходящих точек не найдено!""")

Точка трансформации: Hg9
Точка качества дома: Hg8
Точка сезонной ци: Lu11
Точки трансформации сезонной ци: Lu5
Наиболее эффективная точка для питания: Hg9


In [23]:
################### проходим по слоям без учёта карты пациента ########################

for elem in ['Kid+Ht', 'Gb+Th', 'Sp+Lu', 'St+Co', 'Bl+Si', 'Lv+Hg']:
    if (elem.split('+')[0] in joined) & (elem.split('+')[1] in joined):
        print(f"""Можем использовать слой: {elem}""")
        tochki = []
        for i in df_2.index:
            tochki = []
            for c in df_2.columns:
                tochki.extend(re.findall(f"({elem.split('+')[0]}\d*|{elem.split('+')[1]}\d*)", df_2.loc[i, c]))
            print('\t',i, set(tochki)) 
    else:
        ttt = []
        # for i in df_2.index:
        #     tt = (re.sub(f'({df.loc[df.index[0], "Не используем"]}\d*|{df.loc[df.index[1], "Не используем"]}\d*)', '', df_3.loc[i, df_3.columns[0]]))
        #     tt = tt.replace(',', '').strip()
        #     ttt.append(tt)
        #     print(f"""{i}: {tt}""")

        # ttt = [x for x in ttt if len(x) > 0]
        
        if ttt:
            print(f"""Наиболее эффективная точка для питания: {ttt[0]}""")
        else:
            print(f"""Подходящих точек не найдено!""")

Подходящих точек не найдено!
Можем использовать слой: Gb+Th
	 Точка трансформации {'Th3', 'Gb43'}
	 Точка качества дома {'Th6', 'Gb41'}
	 Точка сезонной ци set()
	 Точки трансформации сезонной ци set()
Подходящих точек не найдено!
Можем использовать слой: St+Co
	 Точка трансформации {'Co11'}
	 Точка качества дома {'Co1'}
	 Точка сезонной ци {'St36'}
	 Точки трансформации сезонной ци {'St41'}
Можем использовать слой: Bl+Si
	 Точка трансформации {'Bl67'}
	 Точка качества дома {'Bl66'}
	 Точка сезонной ци {'Bl67', 'Si8'}
	 Точки трансформации сезонной ци {'Bl40', 'Si5'}
Подходящих точек не найдено!
